<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220107_sm_rf_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
import warnings
from sklearn.ensemble import RandomForestClassifier

In [71]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [72]:
warnings.filterwarnings(action='ignore') 
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]

In [73]:
# cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq5_2', 'bq19_1', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']
#bq5_2, bq19_1, bq31(처리필요)
cols = ['bq4_1a', 'bq4_1b', 'bq4_1c', 'bq30', 'bq31', 'bq32', 'bq33', 'bq34', 'bq38_1']

In [74]:
train1['bq31']

0                                   없다
1                   매타기, 드라이버, 가위, CAD
2                건축설계표, 오토캐드, 파이프, 보일러
3                          토탈스테이션, SZP
4                         캐드, 엑셀, 건축도면
                     ...              
9481             한글, 편집프로그램, 포토샵, 일러스트
9482    AUTO CAD , CATIA, 전문 특수정비도구(툴)
9483                            볼트, 리벳
9484                               컴퓨터
9485                          차량관리 시스템
Name: bq31, Length: 9486, dtype: object

In [75]:
def text_to_list(text:str):
  result = []
  for i in text.split(','):
    result.append(i)
  return result

train1['bq31'] = train1['bq31'].map(text_to_list)

In [76]:
def list_truncate(tool:list):
  if len(tool) > 3:
    return tool[:3]
  elif len(tool) == 3:
    return tool
  elif len(tool) == 2:
    return tool+['0']
  elif len(tool) == 1:
    return tool+['0', '0']
  elif len(tool) == 0:
    return ['0', '0', '0']

train1['bq31'] = train1['bq31'].map(list_truncate)

def list_to_feature1(tool:list):
  return tool[0]

def list_to_feature2(tool:list):
  return tool[1]

def list_to_feature3(tool:list):
  return tool[2]

feature1 = train1['bq31'].map(list_to_feature1)
feature2 = train1['bq31'].map(list_to_feature2)
feature3 = train1['bq31'].map(list_to_feature3)

train1['feature1'] = feature1
train1['feature2'] = feature2
train1['feature3'] = feature3

train1[['feature1', 'feature2', 'feature3']]

,feature1,feature2,feature3
0,없다,0,0
1,매타기,드라이버,가위
2,건축설계표,오토캐드,파이프
3,토탈스테이션,SZP,0
4,캐드,엑셀,건축도면
...,...,...,...
9481,한글,편집프로그램,포토샵
9482,AUTO CAD,CATIA,전문 특수정비도구(툴)
9483,볼트,리벳,0
9484,컴퓨터,0,0


In [77]:
cols = train1.columns.drop(['bq31', 'bq5_2', 'bq19_1'])

In [78]:
# cols.remove('bq31')
# cols = cols + ['feature1', 'feature2', 'feature3']
train1 = train1[cols]

In [79]:
train1

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,...,bq18_5,bq18_6,bq18_7,bq19,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode,feature1,feature2,feature3
0,0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,...,2,3,3,2,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,,2200,825101,없다,0,0
1,1,4,5,4,5,3,4,3,4,3,4,1,,1,,1,,1,,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,,2,...,2,2,4,2,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,,,,2,1,38,4,건축공학,1,1,1,,,2400,140204,매타기,드라이버,가위
2,2,3,4,3,4,3,4,5,6,4,5,1,,1,,3,4,3,4,3,4,1,,3,4,3,4,1,,3,4,3,4,3,4,3,4,3,4,3,...,2,2,4,2,3,2,5,40.0,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,,,,2,1,50,4,건축공학,1,1,1,4000,,2400,140204,건축설계표,오토캐드,파이프
3,3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,...,2,2,3,1,4,2,6,30.0,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,무,지적공무원,,4,1,42,4,환경학과,1,1,1,7000,,3500,140601,토탈스테이션,SZP,0
4,4,4,5,3,4,3,4,4,5,3,4,1,,1,,1,,1,,3,4,2,2,3,4,3,4,1,,3,4,3,4,3,4,3,4,1,,2,...,2,2,4,2,3,2,3,40.0,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,,,,2,1,51,4,건축공학,1,1,1,4000,,2500,140204,캐드,엑셀,건축도면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,5,2,4,3,3,2,2,2,3,2,3,3,3,2,3,2,3,3,4,4,5,4,5,3,4,2,3,3,4,2,2,2,2,2,2,3,5,1,...,3,2,3,3,2,3,1,10.0,2,1,2,2,2,2,2,2,4,4,4,4,3,없다,없다,마케팅업무,없다,2,1,50,4,산업디자인,1,1,1,5200,,1800,411301,한글,편집프로그램,포토샵
9482,9482,5,5,5,5,5,5,3,4,4,5,5,5,4,4,5,5,4,5,4,5,4,5,5,5,4,5,4,5,5,4,2,2,3,4,4,5,5,5,3,...,1,2,4,4,3,3,5,20.0,2,2,1,2,2,2,2,2,4,4,3,4,3,없다,없다,없다,없다,2,1,37,4,우주항공공학,1,1,1,4000,,3000,151105,AUTO CAD,CATIA,전문 특수정비도구(툴)
9483,9483,3,3,4,6,3,3,4,5,4,5,3,4,3,4,1,,2,2,3,3,3,4,3,4,3,4,3,4,3,3,5,7,5,7,4,6,1,,4,...,4,3,3,3,2,3,1,0.0,2,2,2,2,2,2,2,2,3,3,4,3,1,건설 노가대,없다,없다,없다,2,1,32,2,기계,1,4,2,2700,,1500,701101,볼트,리벳,0
9484,9484,3,5,3,5,4,5,3,4,3,5,4,6,4,5,4,5,4,6,4,6,3,4,3,5,3,4,2,3,4,6,3,3,3,4,3,3,4,5,4,...,3,2,3,3,2,2,5,20.0,2,1,1,2,1,2,1,1,3,4,4,3,2,없다,없다,모름,,2,2,40,4,문헌정보학,1,1,1,6800,,2500,25402,컴퓨터,0,0


In [80]:
for col in train1.columns:
    train1[col].replace(' ', '0', inplace=True)

In [81]:
train1

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,...,bq18_5,bq18_6,bq18_7,bq19,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,knowcode,feature1,feature2,feature3
0,0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,...,2,3,3,2,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,0,2200,825101,없다,0,0
1,1,4,5,4,5,3,4,3,4,3,4,1,0,1,0,1,0,1,0,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,0,2,...,2,2,4,2,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,0,0,0,2,1,38,4,건축공학,1,1,1,0,0,2400,140204,매타기,드라이버,가위
2,2,3,4,3,4,3,4,5,6,4,5,1,0,1,0,3,4,3,4,3,4,1,0,3,4,3,4,1,0,3,4,3,4,3,4,3,4,3,4,3,...,2,2,4,2,3,2,5,40.0,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,0,0,0,2,1,50,4,건축공학,1,1,1,4000,0,2400,140204,건축설계표,오토캐드,파이프
3,3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,...,2,2,3,1,4,2,6,30.0,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,무,지적공무원,0,4,1,42,4,환경학과,1,1,1,7000,0,3500,140601,토탈스테이션,SZP,0
4,4,4,5,3,4,3,4,4,5,3,4,1,0,1,0,1,0,1,0,3,4,2,2,3,4,3,4,1,0,3,4,3,4,3,4,3,4,1,0,2,...,2,2,4,2,3,2,3,40.0,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,0,0,0,2,1,51,4,건축공학,1,1,1,4000,0,2500,140204,캐드,엑셀,건축도면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,9481,3,5,2,4,3,3,2,2,2,3,2,3,3,3,2,3,2,3,3,4,4,5,4,5,3,4,2,3,3,4,2,2,2,2,2,2,3,5,1,...,3,2,3,3,2,3,1,10.0,2,1,2,2,2,2,2,2,4,4,4,4,3,없다,없다,마케팅업무,없다,2,1,50,4,산업디자인,1,1,1,5200,0,1800,411301,한글,편집프로그램,포토샵
9482,9482,5,5,5,5,5,5,3,4,4,5,5,5,4,4,5,5,4,5,4,5,4,5,5,5,4,5,4,5,5,4,2,2,3,4,4,5,5,5,3,...,1,2,4,4,3,3,5,20.0,2,2,1,2,2,2,2,2,4,4,3,4,3,없다,없다,없다,없다,2,1,37,4,우주항공공학,1,1,1,4000,0,3000,151105,AUTO CAD,CATIA,전문 특수정비도구(툴)
9483,9483,3,3,4,6,3,3,4,5,4,5,3,4,3,4,1,0,2,2,3,3,3,4,3,4,3,4,3,4,3,3,5,7,5,7,4,6,1,0,4,...,4,3,3,3,2,3,1,0.0,2,2,2,2,2,2,2,2,3,3,4,3,1,건설 노가대,없다,없다,없다,2,1,32,2,기계,1,4,2,2700,0,1500,701101,볼트,리벳,0
9484,9484,3,5,3,5,4,5,3,4,3,5,4,6,4,5,4,5,4,6,4,6,3,4,3,5,3,4,2,3,4,6,3,3,3,4,3,3,4,5,4,...,3,2,3,3,2,2,5,20.0,2,1,1,2,1,2,1,1,3,4,4,3,2,없다,없다,모름,0,2,2,40,4,문헌정보학,1,1,1,6800,0,2500,25402,컴퓨터,0,0


In [82]:
X = train1[train1.columns[1:]]
y = train1['knowcode']

In [87]:
X = X.drop('knowcode', axis=1)

In [88]:
X

,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,aq5_2,aq6_1,aq6_2,aq7_1,aq7_2,aq8_1,aq8_2,aq9_1,aq9_2,aq10_1,aq10_2,aq11_1,aq11_2,aq12_1,aq12_2,aq13_1,aq13_2,aq14_1,aq14_2,aq15_1,aq15_2,aq16_1,aq16_2,aq17_1,aq17_2,aq18_1,aq18_2,aq19_1,aq19_2,aq20_1,aq20_2,...,bq18_4,bq18_5,bq18_6,bq18_7,bq19,bq20,bq21,bq22,bq23,bq24_1,bq24_2,bq24_3,bq24_4,bq24_5,bq24_6,bq24_7,bq24_8,bq25,bq26,bq27,bq28,bq29,bq30,bq32,bq33,bq34,bq35,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3,feature1,feature2,feature3
0,3,3,3,3,3,3,4,4,3,4,3,3,2,2,2,2,2,2,3,3,3,3,4,4,3,3,3,2,3,3,5,6,5,6,4,5,2,2,5,6,...,3,2,3,3,2,2,3,5,10.0,1,1,1,2,2,2,2,2,3,3,4,3,2,없다,없다,건설현장 노무직,없다,4,1,52,2,실업,1,1,1,4000,0,2200,없다,0,0
1,4,5,4,5,3,4,3,4,3,4,1,0,1,0,1,0,1,0,2,3,3,4,3,4,4,5,2,2,3,4,3,4,3,4,3,4,1,0,2,3,...,4,2,2,4,2,3,3,6,50.0,2,2,2,2,2,2,2,2,4,3,4,3,2,없다,0,0,0,2,1,38,4,건축공학,1,1,1,0,0,2400,매타기,드라이버,가위
2,3,4,3,4,3,4,5,6,4,5,1,0,1,0,3,4,3,4,3,4,1,0,3,4,3,4,1,0,3,4,3,4,3,4,3,4,3,4,3,4,...,4,2,2,4,2,3,2,5,40.0,2,2,2,2,2,2,2,2,4,4,4,2,2,없다,0,0,0,2,1,50,4,건축공학,1,1,1,4000,0,2400,건축설계표,오토캐드,파이프
3,3,3,3,3,3,5,4,5,4,6,3,5,3,4,4,5,5,5,4,5,3,4,4,5,3,5,3,4,4,4,3,4,2,2,4,4,4,2,4,3,...,3,2,2,3,1,4,2,6,30.0,2,1,1,2,1,1,1,1,3,4,4,4,4,지적기사,무,지적공무원,0,4,1,42,4,환경학과,1,1,1,7000,0,3500,토탈스테이션,SZP,0
4,4,5,3,4,3,4,4,5,3,4,1,0,1,0,1,0,1,0,3,4,2,2,3,4,3,4,1,0,3,4,3,4,3,4,3,4,1,0,2,3,...,4,2,2,4,2,3,2,3,40.0,2,2,2,2,2,2,2,2,4,3,4,3,3,없다,0,0,0,2,1,51,4,건축공학,1,1,1,4000,0,2500,캐드,엑셀,건축도면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,3,5,2,4,3,3,2,2,2,3,2,3,3,3,2,3,2,3,3,4,4,5,4,5,3,4,2,3,3,4,2,2,2,2,2,2,3,5,1,0,...,4,3,2,3,3,2,3,1,10.0,2,1,2,2,2,2,2,2,4,4,4,4,3,없다,없다,마케팅업무,없다,2,1,50,4,산업디자인,1,1,1,5200,0,1800,한글,편집프로그램,포토샵
9482,5,5,5,5,5,5,3,4,4,5,5,5,4,4,5,5,4,5,4,5,4,5,5,5,4,5,4,5,5,4,2,2,3,4,4,5,5,5,3,4,...,4,1,2,4,4,3,3,5,20.0,2,2,1,2,2,2,2,2,4,4,3,4,3,없다,없다,없다,없다,2,1,37,4,우주항공공학,1,1,1,4000,0,3000,AUTO CAD,CATIA,전문 특수정비도구(툴)
9483,3,3,4,6,3,3,4,5,4,5,3,4,3,4,1,0,2,2,3,3,3,4,3,4,3,4,3,4,3,3,5,7,5,7,4,6,1,0,4,5,...,2,4,3,3,3,2,3,1,0.0,2,2,2,2,2,2,2,2,3,3,4,3,1,건설 노가대,없다,없다,없다,2,1,32,2,기계,1,4,2,2700,0,1500,볼트,리벳,0
9484,3,5,3,5,4,5,3,4,3,5,4,6,4,5,4,5,4,6,4,6,3,4,3,5,3,4,2,3,4,6,3,3,3,4,3,3,4,5,4,4,...,3,3,2,3,3,2,2,5,20.0,2,1,1,2,1,2,1,1,3,4,4,3,2,없다,없다,모름,0,2,2,40,4,문헌정보학,1,1,1,6800,0,2500,컴퓨터,0,0


In [89]:
from sklearn.preprocessing import LabelEncoder

for col in X.columns:
    if col == 'ID':
        continue
    try:
        X[col] = X[col].map(int)
    except:
        encoder = LabelEncoder()
        X[col] = X[col].map(str)
        X[col] = encoder.fit_transform(X[col])

In [ ]:
X

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True)

In [91]:
model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)

In [92]:
model.fit(X_train, y_train)

RandomForestClassifier(n_jobs=8, random_state=123456)

In [93]:
model.score(X_test, y_test)

0.5068493150684932